In [1]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
import unicodedata
import unidecode
import numpy as np
import requests

In [2]:
def ReadDirections(file):
    df_entrada = pd.read_csv(file,sep=';', engine='python', encoding='latin1')
    df = df_entrada.copy()
    return df
df_r = ReadDirections('Directorio.csv')

In [3]:
df_r.head()

,COD_VIA,VIA_CLASE,VIA_PAR,VIA_NOMBRE,VIA_NOMBRE_ACENTOS,DISTRITO,BARRIO,IMPAR_MIN,IMPAR_MAX,PAR_MIN,PAR_MAX
0,31001337,AUTOVÍA,NaN,A-1,A-1,16.0,6.0,NaN,NaN,10000.0,13000.0
1,31001336,AUTOVÍA,NaN,A-2,A-2,4.0,4.0,NaN,NaN,NaN,NaN
2,31001336,AUTOVÍA,NaN,A-2,A-2,5.0,2.0,NaN,NaN,NaN,NaN
3,31001336,AUTOVÍA,NaN,A-2,A-2,15.0,5.0,NaN,NaN,5000.0,5000.0
4,31001336,AUTOVÍA,NaN,A-2,A-2,15.0,6.0,NaN,NaN,5000.0,5000.0


In [4]:
print(len(df_r))
df_r.columns

10770


Index(['COD_VIA', 'VIA_CLASE', 'VIA_PAR', 'VIA_NOMBRE', 'VIA_NOMBRE_ACENTOS',
       'DISTRITO', 'BARRIO', 'IMPAR_MIN', 'IMPAR_MAX', 'PAR_MIN', 'PAR_MAX'],
      dtype='object')

In [9]:
# Dataframe con únicamente las calles:
def CleanDataSetDirections (df):
    df['VIA_CLASE'] = df.VIA_CLASE.apply(lambda x: x if not pd.isnull(x) else '')
    df['VIA_PAR'] = df.VIA_PAR.apply(lambda x: x if not pd.isnull(x) else '')
    df['VIA_NOMBRE'] = df.VIA_NOMBRE.apply(lambda x: x if not pd.isnull(x) else '')
    dfnew = pd.DataFrame(columns=['DIRECTION'])
    dfnew['DIRECTION'] = df['VIA_CLASE'].astype(str) + ' ' + df['VIA_PAR'].astype(str) + ' ' + df['VIA_NOMBRE'].astype(str)
    df = dfnew.drop_duplicates()
    return df

df_calles = CleanDataSetDirections(df_r)
df_calles.head(14)

,DIRECTION
0,AUTOVÍA A-1
1,AUTOVÍA A-2
12,AUTOVÍA A-3
20,AUTOVÍA A-4
22,AUTOVÍA A-42
31,AUTOVÍA A-5
38,AUTOVÍA A-6
43,CALLE DEL ABAD JUAN CATALAN
44,CALLE DE LA ABADA
46,CALLE DE LOS ABADES


def Strip_Accents(accented_directions):
    return ''.join(direction for direction in unidecode.unidecode(accented_directions))
Strip_Accents(df_calles)

In [10]:
def get_gps (adress):
    # cómo hago lo de la API KEY?
    key = 'f983690f78ea42d0801a8fa9b349406d'
    geocoder = OpenCageGeocode(key)
    results = geocoder.geocode(adress)
    return {
        "calle": adress, 
        "latitud" : results[0]['geometry']['lat'], 
        "longitud" : results[0]['geometry']['lng']
    }

In [11]:
def returnDic (df):
    direcciones = []
    for calle in df:
        try: 
            query = get_gps(calle)
            # print(query)
            direcciones.append(query)
        except: 
            print('fallo')
    dic = pd.DataFrame(direcciones)        
    return dic

returnDic (df_calles['DIRECTION'][55:60])

,calle,latitud,longitud
0,CALLE DE ACHURI,40.428671,-3.641780
1,CALLE DE ACONCAGUA,40.468712,-3.631307
2,CALLE DEL ACONITO,40.468682,-3.705484
3,CALLE DE LA ACUARELA,40.287968,-3.754584
4,CALLE DE ACUARIO,40.474705,-3.579315
